# Data Cleaning and Exploratory Data Analysis

## Loading the data

In [1]:
pre = "/Users/patrickokwir/Desktop/Lighthouse-data-notes/Jack-Talk-Surveys/datasets/CASE STUDY - Livestream pre FY23.csv"
post = "/Users/patrickokwir/Desktop/Lighthouse-data-notes/Jack-Talk-Surveys/datasets/CASE STUDY - Livestream post FY23.csv"

In [3]:
# load df from csv file
library(readr)

df_pre <- read.csv(pre, header = TRUE, sep = ",")
df_post <- read.csv(post, header = TRUE, sep = ",")

In [4]:
#head df_pre
head(df_pre)

,Finished,RecordedDate,Q1_a,Q2_a,Q3_a,Q4_a,id
,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,TRUE,2022-09-21 13:13,,,,,
2,TRUE,2022-09-21 13:14,Strongly disagree,Agree,Somewhat agree,Agree,O_A_09_L
3,TRUE,2022-09-21 13:14,Agree,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,A_N_02_8
4,TRUE,2022-09-21 13:15,Strongly disagree,Strongly disagree,Somewhat agree,Neither agree nor disagree,M_U_07_1
5,TRUE,2022-09-21 13:15,Somewhat disagree,Disagree,Agree,Somewhat agree,A_E_06_A
6,TRUE,2022-09-21 13:16,Neither agree nor disagree,Agree,Agree,Agree,Y_O_04_1


In [5]:
head(df_post)

,Finished,RecordedDate,Q1_b,Q2_b,Q3_b,Q4_b,Q5,Q6,Q7,Q8,id
,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,TRUE,2022-09-21 13:55,Neither agree nor disagree,Somewhat agree,Agree,Somewhat agree,Somewhat agree,Somewhat agree,Neither agree nor disagree,5,A_E_06_A
2,TRUE,2022-09-21 13:57,Somewhat agree,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,Agree,Agree,,5,I_K_10_1
3,TRUE,2022-09-21 13:57,Agree,Disagree,Somewhat agree,Agree,Agree,Agree,Neither agree nor disagree,7,A_N_09_D
4,TRUE,2022-09-21 13:58,Agree,Disagree,Strongly agree,Strongly agree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,5,L_T_12_A
5,FALSE,2022-09-24 13:14,,,,,,,,NA,
6,FALSE,2022-09-24 13:16,,,,,,,,NA,


### Data cleaning df_pre

In [7]:
# remove rows from df_pre where column Finished = False
# count rows where column Finished = False
count_false_finished <- sum(df_pre$Finished == FALSE)
count_false_finished

[1] 2878

In [9]:
# Remove rows where the user did not finish the survey
df_pre <- df_pre[df_pre$Finished != FALSE, ]
count_false_finished <- nrow(df_pre[df_pre$Finished == FALSE, ])
count_true_finished <- nrow(df_pre[df_pre$Finished != FALSE, ])

count_false_finished
count_true_finished

[1] 0

[1] 8047

In [12]:
# check for missing values
missing_values <- is.na(df_pre)
# count the number of missing values in each column
colSums(missing_values)

Finished RecordedDate         Q1_a         Q2_a         Q3_a         Q4_a 
           0            0            0            0            0            0 
          id 
           0

In [16]:
# check for duplicates 
duplicates <- duplicated(df_pre)

# find duplicates
has_duplicates <- any(duplicates)
print(has_duplicates)

# count duplicates
sum(duplicates)

# return duplicated rows
df_pre[duplicates, ]

# duplicates seem to be coming from where there is no id and didnt answer any of the questions. 


[1] TRUE


[1] 191

,Finished,RecordedDate,Q1_a,Q2_a,Q3_a,Q4_a,id
,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
40,TRUE,2022-10-03 9:14,,,,,
41,TRUE,2022-10-03 9:14,,,,,
45,TRUE,2022-10-03 9:14,,,,,
357,TRUE,2022-10-12 16:59,,,,,
363,TRUE,2022-10-12 16:59,,,,,
378,TRUE,2022-10-12 17:02,,,,,
416,TRUE,2022-10-12 17:16,,,,,
422,TRUE,2022-10-12 17:17,,,,,
423,TRUE,2022-10-12 17:17,,,,,


In [20]:
# remove that subset of data from the dataset
df_pre <- df_pre[!duplicated(df_pre), ]

In [24]:
# count rows where id is null/blank
count_blank_id <- sum(nchar(df_pre$id) == 0)
count_blank_id

[1] 245

In [52]:
blank_id <- df_pre[nchar(df_pre$id) == 0, ]
head(blank_id)

Finished,RecordedDate,Q1_a,Q2_a,Q3_a,Q4_a,id
<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [28]:
#remove that subset of data from the dataset
df_pre <- df_pre[nchar(df_pre$id) > 0, ]

In [30]:
head(df_pre)

,Finished,RecordedDate,Q1_a,Q2_a,Q3_a,Q4_a,id
,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,TRUE,2022-09-21 13:14,Strongly disagree,Agree,Somewhat agree,Agree,O_A_09_L
3,TRUE,2022-09-21 13:14,Agree,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,A_N_02_8
4,TRUE,2022-09-21 13:15,Strongly disagree,Strongly disagree,Somewhat agree,Neither agree nor disagree,M_U_07_1
5,TRUE,2022-09-21 13:15,Somewhat disagree,Disagree,Agree,Somewhat agree,A_E_06_A
6,TRUE,2022-09-21 13:16,Neither agree nor disagree,Agree,Agree,Agree,Y_O_04_1
7,TRUE,2022-09-21 13:16,Agree,Agree,Somewhat agree,Somewhat agree,A_Y_06_A


In [51]:
#check for blank values
has_blank_rows <- apply(df_pre, 1, function(row) any(is.na(1) | row == ""))
head(df_pre[has_blank_rows,])


,Finished,RecordedDate,Q1_a,Q2_a,Q3_a,Q4_a,id
,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
333,TRUE,2022-10-12 16:53,Agree,Somewhat agree,Neither agree nor disagree,,H_N_02_S
439,TRUE,2022-10-12 17:18,Somewhat disagree,Somewhat agree,Neither agree nor disagree,,N_O_12_N
807,TRUE,2022-10-19 18:09,Neither agree nor disagree,Strongly disagree,Strongly agree,,A_N_08_3
1254,TRUE,2022-11-21 18:25,Neither agree nor disagree,Neither agree nor disagree,,,E_U_02_4
1311,TRUE,2022-11-30 7:39,Neither agree nor disagree,,,,T_Q_03_T
1579,TRUE,2023-01-11 13:34,Neither agree nor disagree,Neither agree nor disagree,,Agree,N_A_11_W


In [35]:
# Check for rows where Q1_a, Q2_a, Q3_a, and Q4_a are all blank
blank_rows <- apply(df_pre[, c("Q1_a", "Q2_a", "Q3_a", "Q4_a")], 1, function(row) all(is.na(row) | row == ""))
sum(blank_rows)

# 39 observations where the questions were not filled out

[1] 39

In [36]:
#Remove that subset of data from the original dataset
df_pre <- df_pre[!blank_rows, ]
head(df_pre)

,Finished,RecordedDate,Q1_a,Q2_a,Q3_a,Q4_a,id
,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,TRUE,2022-09-21 13:14,Strongly disagree,Agree,Somewhat agree,Agree,O_A_09_L
3,TRUE,2022-09-21 13:14,Agree,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,A_N_02_8
4,TRUE,2022-09-21 13:15,Strongly disagree,Strongly disagree,Somewhat agree,Neither agree nor disagree,M_U_07_1
5,TRUE,2022-09-21 13:15,Somewhat disagree,Disagree,Agree,Somewhat agree,A_E_06_A
6,TRUE,2022-09-21 13:16,Neither agree nor disagree,Agree,Agree,Agree,Y_O_04_1
7,TRUE,2022-09-21 13:16,Agree,Agree,Somewhat agree,Somewhat agree,A_Y_06_A


In [37]:
#check for blank values
has_blank_rows <- apply(df_pre, 1, function(row) any(is.na(1) | row == ""))
head(df_pre[has_blank_rows,])

,Finished,RecordedDate,Q1_a,Q2_a,Q3_a,Q4_a,id
,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
333,TRUE,2022-10-12 16:53,Agree,Somewhat agree,Neither agree nor disagree,,H_N_02_S
439,TRUE,2022-10-12 17:18,Somewhat disagree,Somewhat agree,Neither agree nor disagree,,N_O_12_N
807,TRUE,2022-10-19 18:09,Neither agree nor disagree,Strongly disagree,Strongly agree,,A_N_08_3
1254,TRUE,2022-11-21 18:25,Neither agree nor disagree,Neither agree nor disagree,,,E_U_02_4
1311,TRUE,2022-11-30 7:39,Neither agree nor disagree,,,,T_Q_03_T
1579,TRUE,2023-01-11 13:34,Neither agree nor disagree,Neither agree nor disagree,,Agree,N_A_11_W


In [39]:
# Describe summary of df_pre
summary(df_pre)

 Finished       RecordedDate           Q1_a               Q2_a          
 Mode:logical   Length:7572        Length:7572        Length:7572       
 TRUE:7572      Class :character   Class :character   Class :character  
                Mode  :character   Mode  :character   Mode  :character  
     Q3_a               Q4_a                id           
 Length:7572        Length:7572        Length:7572       
 Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character  

In [50]:
# get value counts for each column

# Install and load the dplyr package if not already installed
library(dplyr)

for (col_name in names(df_pre)) {
  unique_values <- unique(na.omit(df_pre[[col_name]]))
  cat("Unique non-blank values for column:", col_name, "\n")
  print(unique_values)
  cat("\n")
}

Unique non-blank values for column: Finished 
[1] TRUE

Unique non-blank values for column: RecordedDate 
   [1] "2022-09-21 13:14" "2022-09-21 13:15" "2022-09-21 13:16"
   [4] "2022-09-21 13:17" "2022-09-21 13:18" "2022-09-21 13:19"
   [7] "2022-09-21 13:21" "2022-10-03 9:10"  "2022-10-03 9:12" 
  [10] "2022-10-03 9:13"  "2022-10-03 9:14"  "2022-10-03 9:15" 
  [13] "2022-10-03 9:16"  "2022-10-03 9:17"  "2022-10-03 9:18" 
  [16] "2022-10-03 9:19"  "2022-10-03 9:20"  "2022-10-03 9:21" 
  [19] "2022-10-03 9:22"  "2022-10-03 9:23"  "2022-10-03 9:24" 
  [22] "2022-10-03 9:25"  "2022-10-03 9:26"  "2022-10-03 9:27" 
  [25] "2022-10-03 9:28"  "2022-10-03 9:29"  "2022-10-03 9:30" 
  [28] "2022-10-03 9:31"  "2022-10-03 9:33"  "2022-10-03 9:35" 
  [31] "2022-10-03 9:37"  "2022-10-03 9:44"  "2022-10-03 10:11"
  [34] "2022-10-03 10:17" "2022-10-03 10:18" "2022-10-03 10:19"
  [37] "2022-10-03 10:25" "2022-10-03 11:39" "2022-10-07 15:25"
  [40] "2022-10-07 15:26" "2022-10-07 15:27" "2022-10-07 15:28